In [1]:
import polars as pl
from polars import col
pl.__version__

'0.13.24'

### Load 70 required CMS services
I manually put these in a csv file. Data from here:
https://www.cms.gov/icd10m/version40-fullcode-cms/fullcode_cms/P0002.html

In [2]:
cms_codes = pl.read_csv('70_shoppables_with_medicare.csv', dtype = {'code': int})
cms_codes.head()

description,short_description,facility_rate,code,kind
str,str,f64,i64,str
"""Psychotherapy, 30 min""","""Psytx w pt 30 minutes""",68.87,90832,"""Evaluation and management services"""
"""Psychotherapy, 45 min""","""Psytx w pt 45 minutes""",90.67,90834,"""Evaluation and management services"""
"""Psychotherapy, 60 min""","""Psytx w pt 60 minutes""",132.89,90837,"""Evaluation and management services"""
"""Family psychotherapy, not including patient, 50 min""","""Family psytx w/o pt 50 min""",97.59,90846,"""Evaluation and management services"""
"""Family psychotherapy, including patient, 50 min""","""Family psytx w/pt 50 min""",101.4,90847,"""Evaluation and management services"""


### Reading in the ranged values

From glancing at the 70 CMS required codes we can see that they are **all numeric.** This reduces the cases we need to look at.

These codes are only CPT/HCPCS and DRG codes. None of them have modifier suffixes, like `-G1`, which slightly change the code/price typically. Nor do they include NDC codes, which are drug-specific.

Here's how we'll do the regexp filtering on the dataframe.

1. If there's a numerical code range (regexpr) `\d{5}-\d{5}`, we'll need to expand it. Capture it with the following regex (`\b` denotes a word bounday) `\b\d{5}-\d{5}\b`
2. If there's a numerical code range `\d{2,3}-\d{3}`, we'll need to expand it: `\b\d{3}-\d{3}\b`
3. There are still cases of NDC codes (ten digit codes which indicate a specific drug) disguised as case (1). Almost all of those cases also have internal revenue code `0000` so that's how we'll filter those out.

### Use a lazy dataframe to pre-filter and save memory

`polars` is helpful in that it can save memory by filtering the CSV file _on read-in_, which means we can load the entire thing into memory instead of having to do out-of-memory analysis.

In [4]:
ignored_payers = '|'.join([
    'maximum',
    'minimum',
    '^max$',
    '^min$',
])

codes_included = '|'.join([
    '\\b\d{5}\\b', #..................... 65580 BUT NOT 65880-12
    '\\b\d{3}\\b', #..................... 312 and BUT NOT 12-312 or 123-12
    '\\b\d{5}-\s?\d{5}\\b', #............ 10012-69999
    '\\b\d{3}-\s?\d{3}\\b', #............ 123-127
])

codes_excluded = '|'.join([
    '\\bndc\\b', #........................ NDC <anything>
    '\\beapg\\b', #....................... EAPG <anything>
    '\d+-\d+-\d', #....................... 1234-1234-12 (any 10-digit string with two dashes)
    '\d{10}', #........................... 10-digit str
    '\\bgrams\\b', #...................... usually a drug
])

words_to_erase = '|'.join([
    '\\b[a-zA-Z][0-9]{4}\\b', #........... Q1234
    '\\b[0-9]{4}[a-zA-Z]\\b', #........... 1234Q
    '\\b(\w+-){2,}\w\\b', #............... XXX-XX-XXXXX
    '(\\b\d{1,2}|\\b\d{4,})-\d{3}\\b', #.. 1-123, 2-123, 12345-123
    '\\b\d{3}-(\d{1,2}\\b|\d{4,}\\b)', #.. 123-12, 123-1234
    '(\\b\d{1,4}|\\b\d{6,})-\d{5}\\b', #.. 1-12345, 1234-12345
    '\\b\d{5}-(\d{1,4}\\b|\d{6,}\\b)', #.. 12345-123, 12345-1234, 12345-123456
    '[a-zA-Z]+\s?[\d]{1,2}\\b', #......... V12, Version 91
    '[FY|FY\\s|\\s]20[12][0-9]\\b', #..... FY 2021, FY2020, FY 2012, 2012
    '\\b\d{5}-?\w{2}\\b', #............... 13711-GW or 12932-12
    '\\b\d{3}-?\d{1}\\b', #............... 923-1
    '\\b00\d{3}\\b', #.................... 00416 
    '\\b\$?\d+\.\d\d\\b', #............... $12.00, 2.00
    
])

dtype = {'cms_certification_num':str,
         'code':str,
         'internal_revenue_code':str,
         'units':str,
         'code_disambiguator':str}

ql = (pl.scan_csv('prices.csv', dtype = dtype, encoding='utf8-lossy', low_memory = True)      
    
      .filter(
          (col('price') > 0) &
          (col('code').str.contains(fr'{codes_included}')) &
          ~(col('internal_revenue_code') == '0000') &
          ~col('code').str.to_lowercase().str.contains(fr'{codes_excluded}') &
          ~col('payer').str.to_lowercase().str.contains(fr'{ignored_payers}')
      )
      
      .with_column(
          (col('code')
           
           # erase bad strings
           .str.replace_all(fr'{words_to_erase}', ' ')
           
           # remove any non-digit characters except dashes and replace with spaces
           .str.replace_all(r'[^\d-]+', ' ')
           .str.strip()
           
           # replace any dashes with space by ordinary dashes, 
           .str.replace_all(r'\s?-\s?', '-')
           
           # split into a string based on spacing
           .str.split(' ')
           .alias('code_extracted'))
      )
      
      .explode('code_extracted')
      
      .with_column(
          # remove leading dashes and trailing dashes
          col('code_extracted').str.replace_all(r'^(-)+|-+$', ' ').str.strip()
      )
      
      .filter(col('code_extracted').str.lengths() > 0)
     )

In [6]:
%%time
df = ql.collect()

CPU times: user 10min 9s, sys: 3min 37s, total: 13min 46s
Wall time: 5min 24s


In [7]:
df_n = df.filter(~col('code_extracted').str.contains('-'))
df_r = df.filter(col('code_extracted').str.contains('-'))

In [8]:
# "normal" dataframe -- no ranges (indicated by dash)
df_n = (df_n
        .with_column(
            col('code_extracted').cast(pl.Int64))
        .filter(pl.col('code_extracted').is_in(cms_codes['code']))
       )

In [9]:
cms_codes_set = set(cms_codes['code'])

def get_valid_codes(struct: dict) -> list:
    code_list   = set(range(struct['init'], struct['final'] + 1))
    valid_codes =  list(set.intersection(cms_codes_set, code_list))
    return valid_codes if valid_codes else [0]

In [10]:
%%time
# "ranged" dataframe -- codes with ranges
# Although it doesn't look elegant, this code runs faster than an elegant vectorized solution
# and I'm not sure why.
df_r = (df_r
        .with_column(
            col('code_extracted').str.split('-')
         )
         .with_columns([
             col('code_extracted').arr.first().cast(int).alias('init'),
             col('code_extracted').arr.last().cast(int).alias('final'),
         ])
         .with_column(
             pl.struct(['init', 'final']).apply(get_valid_codes).cast(pl.List(pl.Int64)).alias('code_extracted')
         )
         .drop(['init', 'final'])
         .filter(col('code_extracted').arr.first() != 0)
         .explode('code_extracted')
        )

CPU times: user 4.18 s, sys: 521 ms, total: 4.7 s
Wall time: 4.93 s


## Handling the code ranges

In [11]:
# Finally, put the two dataframes together
df = pl.concat([df_r, df_n])

In [12]:
print(f'Total rows (prices): {len(df)}')
print(df.head())

Total rows (prices): 1727720
shape: (5, 10)
┌────────────┬─────────┬────────┬────────────┬─────┬────────────┬────────┬────────────┬────────────┐
│ cms_certif ┆ payer   ┆ code   ┆ internal_r ┆ ... ┆ inpatient_ ┆ price  ┆ code_disam ┆ code_extra │
│ ication_nu ┆ ---     ┆ ---    ┆ evenue_cod ┆     ┆ outpatient ┆ ---    ┆ biguator   ┆ cted       │
│ m          ┆ str     ┆ str    ┆ e          ┆     ┆ ---        ┆ f64    ┆ ---        ┆ ---        │
│ ---        ┆         ┆        ┆ ---        ┆     ┆ str        ┆        ┆ str        ┆ i64        │
│ str        ┆         ┆        ┆ str        ┆     ┆            ┆        ┆            ┆            │
╞════════════╪═════════╪════════╪════════════╪═════╪════════════╪════════╪════════════╪════════════╡
│ 020017     ┆ Aetna   ┆ CPT/HC ┆ NONE       ┆ ... ┆ UNSPECIFIE ┆ 43943. ┆ NONE       ┆ 93452      │
│            ┆ PPO     ┆ PC 934 ┆            ┆     ┆ D          ┆ 0      ┆            ┆            │
│            ┆         ┆ 51-934 ┆            ┆ 

### Import the hospitals data

In [13]:
hospitals = pl.read_csv('hospitals.csv', dtype = dtype)

## Looking at insurer prices

In [15]:
# Look at insurer prices only
df_insurer = (df
              .filter(col('payer') != 'GROSS CHARGE')
              .filter(col('payer') != 'CASH PRICE')
              .groupby(['cms_certification_num', 'code_extracted'])
              .agg(col('price').mean().alias('mean_price'))
             )

In [21]:
hosp_top_10 = (df_insurer
               .groupby('code_extracted')
               .agg([
                   col(['cms_certification_num', 'mean_price']).sort_by('mean_price').tail(10),
                   col('mean_price').sort_by('mean_price').tail(10).rank('dense', reverse = True).alias('rank'),
                   pl.count(),
               ])
               .explode(['cms_certification_num', 'mean_price', 'rank'])
               .join(hospitals, on = 'cms_certification_num')
               .join(cms_codes, right_on = 'code', left_on = 'code_extracted')
               .select(['code_extracted', 'rank', 'count', 'description', 'mean_price', 'name', 'city', 'state', 'cms_certification_num', 'chargemaster_url'])
               .sort(['code_extracted', 'mean_price'])
              )

In [22]:
# Show the top 10 hospitals for each code, and rank
hosp_top_10.sort('count').head(10)

code_extracted,rank,count,description,mean_price,name,city,state,cms_certification_num,chargemaster_url
i64,u32,u32,str,f64,str,str,str,str,str
59610,8,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",11482.0,"""CLEVELAND CLINIC""","""CLEVELAND""","""OH""","""360180""","""https://my.clevelandclinic.org/-/scassets/files/org/patients-visitors/hospital-standard-charge-files/340714585_cleveland-clinic-main-campus_standardcharges.csv"""
59610,7,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",12295.625,"""PUTNAM COMMUNITY MEDICAL CENTER""","""PALATKA""","""FL""","""100232""","""https://core.secure.ehc.com/src/util/detail-price-list/47-2762362_putnam-community-medical-center_standardcharges.csv"""
59610,6,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",12859.4,"""WEST FLORIDA HOSPITAL""","""PENSACOLA""","""FL""","""100231""","""https://core.secure.ehc.com/src/util/detail-price-list/59-1525468_west-florida-hospital_standardcharges.csv"""
59610,5,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",14526.4,"""TWIN CITIES HOSPITAL""","""NICEVILLE""","""FL""","""100054""","""https://core.secure.ehc.com/src/util/detail-price-list/59-1836808_twin-cities-hospital_standardcharges.csv"""
59610,5,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",14526.4,"""FORT WALTON BEACH MEDICAL CENTER""","""FORT WALTON BEACH""","""FL""","""100223""","""https://core.secure.ehc.com/src/util/detail-price-list/611259833_fort-walton-beach-medical-center_standardcharges.csv"""
59610,5,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",14526.4,"""GULF COAST REGIONAL MEDICAL CENTER""","""PANAMA CITY""","""FL""","""100242""","""https://core.secure.ehc.com/src/util/detail-price-list/620976863_gulf-coast-regional-medical-center_standardcharges.csv"""
59610,4,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",15693.666,"""NORTH FLORIDA REGIONAL MEDICAL CENTER""","""GAINESVILLE""","""FL""","""100204""","""https://core.secure.ehc.com/src/util/detail-price-list/61-1269294_north-florida-regional-medical-center_standardcharges.csv"""
59610,3,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",16913.25,"""PORTSMOUTH REGIONAL HOSPITAL""","""PORTSMOUTH""","""NH""","""300029""","""https://core.secure.ehc.com/src/util/detail-price-list/02-0364103_portsmouth-regional-hospital_standardcharges.csv"""
59610,2,222,"""Routine obstetric care for vaginal delivery after prior cesarean delivery including pre-and post-delivery care""",17694.625,"""PARKLAND MEDICAL CENTER""","""DERRY""","""NH""","""300017""","""https://core.secure.ehc.com/src/util/detail-price-list/020364103_parkland-medical-center_standardcharges.csv"""


In [23]:
print(hosp_top_10['name'].value_counts()[:10].to_pandas().set_index('name').to_markdown())

| name                                  |   counts |
|:--------------------------------------|---------:|
| MARY LANNING HEALTHCARE               |       36 |
| ENGLEWOOD COMMUNITY HOSPITAL          |       23 |
| BLAKE MEDICAL CENTER                  |       23 |
| SONOMA VALLEY HOSPITAL                |       17 |
| NORTH FLORIDA REGIONAL MEDICAL CENTER |       16 |
| PORTSMOUTH REGIONAL HOSPITAL          |       16 |
| ADVENTHEALTH CONNERTON                |       15 |
| TWIN CITIES HOSPITAL                  |       15 |
| GULF COAST REGIONAL MEDICAL CENTER    |       14 |
| FORT WALTON BEACH MEDICAL CENTER      |       13 |


### Organize the data by code and plot

In [71]:
import altair as alt

alt.data_transformers.disable_max_rows()

# https://stackoverflow.com/questions/68134680/how-to-add-more-space-between-y-axis-and-bar-chart-in-altair-visual
# https://stackoverflow.com/questions/57244390/how-to-add-a-subtitle-to-an-altair-generated-chart
# https://www.optumcoding.com/upload/docs/2021%20DRG_National%20Average%20Payment%20Table_Update.pdf
# https://revcycleintelligence.com/news/medicaid-physician-reimbursement-rates-lag-medicare

def dolthub_theme():
    return {
        'config': {
            'legend': {
                'labelLimit': 700,
                'strokeColor': 'gray',
                'fillColor': '#EEEEEE',
                'padding': 10,
            },
        }
    }
alt.themes.register('dolthub_theme', dolthub_theme)# enable the newly registered theme
alt.themes.enable('dolthub_theme')

ThemeRegistry.enable('dolthub_theme')

### Plot the data for a specific code

In [72]:
source = (df
          .filter(col('code_extracted') == 473)
          .groupby('cms_certification_num')
          .agg(
              col('price').mean()
          )
          .join(hospitals, on = 'cms_certification_num')
          .filter(col('state') == 'NE')
          .to_pandas()
         )

chart = (alt.Chart(source)
         .mark_bar()
         .encode(x = 'price:Q', y = alt.Y('name:N', sort = '-x', title = ''))
        )

chart

alt.Chart(...)

In [73]:
source = (df
          .filter(col('payer') == 'GROSS CHARGE')
          .with_column(
              col('price').mean().over(['code_extracted', 'cms_certification_num']).alias('mean_code_price')
          )
          .filter(col('price') < 2*col('mean_code_price'))
          .filter(col('price') > .5*col('mean_code_price'))
          .groupby(['code_extracted', 'cms_certification_num']).agg(col('price').mean())
          .join(cms_codes, left_on = 'code_extracted', right_on = 'code')
          .filter(col('price') > 1)
          .filter(col('price') < 1_000_000)
          .with_column(
              (col('facility_rate')*.72).alias('estimated_medicaid_rate')
          )
          # remove sample to increase number of points
          .sample(10_000)
          .to_pandas())

# MCC = major complications or comorbidities
# CC = complications or comorbidities

payers = (
    alt.Chart(source, 
             )
    .mark_tick(
        opacity = .5,
        size = 10,
    )
    .encode(
        x=alt.X(
            title = 'procedure list price (colored), est. Medicaid price (black) ($)',
            field = 'price',
            scale = alt.Scale(type ='log'),
        ),
        y=alt.Y(
            # title = '',
            field='description',
            sort = cms_codes.sort('kind')['description'].to_list(),
        ),
        color=alt.Color('kind', title ='Procedure type', legend=alt.Legend(symbolOpacity=1)),
    )
)

medicare = (
    alt.Chart(source)
    .mark_point(
        color = 'black',
        filled = True,
        size = 40,
    )
    .encode(
        x=alt.X(
            # field = 'facility_rate',
            field = 'estimated_medicaid_rate',
            scale = alt.Scale(type ='log', domain = (1, 1_000_000))
        ),
        y=alt.Y(
            title = '',
            field='description',
            sort = cms_codes.sort('kind')['description'].to_list(),
        ),
    )
)

chart = payers + medicare

chart = (chart
         .properties(
                 title={
                      "text": ['"List prices" and Medicaid prices'], 
                      "subtitle": ['Hospital list prices for 70 common services (logarithmic scale)',
                                   'Each tick is a different price',
                                   'Estimated Medicaid price as black dot, when available']
                    },
             width = 800,
         )
         .configure_title(
            fontSize = 25,
            anchor = 'middle',
            subtitleFontSize = 20,
            subtitleColor = 'gray',
         ).configure_legend(
             labelFontSize = 13,
             titleFontSize = 13,
             orient = 'top-right',
         ).configure_axisY(
             grid = True
         ).configure_axisLeft(
             labelFontSize = 14,
             labelLimit = 700,
         ).configure_axisBottom(
             titleFontSize = 20,
         )
)

# chart.save('fig_gross.png')

chart

alt.LayerChart(...)

In [74]:
df1 = df.filter(col('payer') == 'GROSS CHARGE')
df2 = df.filter(col('payer') == 'CASH PRICE')
df3 = df.filter(col('payer') != 'GROSS CHARGE').filter(col('payer') != 'CASH PRICE')

In [75]:
df1 = (df1.groupby(['code_extracted']).agg(col('price').mean())).with_column(pl.lit('List Price').alias('payer'))
df2 = (df2.groupby(['code_extracted']).agg(col('price').mean())).with_column(pl.lit('Cash Price').alias('payer'))
df3 = (df3.groupby(['code_extracted']).agg(col('price').mean())).with_column(pl.lit('Insurer Price').alias('payer'))

In [76]:
df4 = (pl.concat([df1,df2,df3])
       .join(cms_codes, left_on = 'code_extracted', right_on = 'code'))

In [77]:
source = df4.groupby('payer').agg(col('price').mean()).to_pandas()

chart = alt.Chart(source).mark_bar(size = 100).encode(
    x=alt.X('payer:N', title = '', sort = '-y'),
    y=alt.Y('price:Q', title = 'average price over all codes ($)'),
    color='payer:N',
).configure_legend(
    orient = 'right'
).properties(
    width = 500,
    title={
      "text": ["List prices, cash prices, and insurance prices"], 
    }
).configure_title(
            fontSize = 20,
            anchor = 'middle',
            subtitleFontSize = 18,
            subtitleColor = 'gray',
)

# chart.save('price_bar_chart.png')

chart

alt.Chart(...)